# Exercice

On vous demande dans cet exercice de programmer un algorithme génétique pour résoudre le problème

du sac à dos 0/1. Vous testerez votre algorithme sur les données fournies lors des séances précédentes. Vous

comparerez statistiquement (donc en faisant de multiples run de votre algorithme, ce qui est indispensable

quand l’aléatoire intervient) les résultats de votre algorithme avec ceux obtenus par les méthodes concurrentes.

Vous pourrez également analyser l’impact des différents paramètres de l’algorithme.

In [1]:
import numpy as np
import os

In [2]:

np.random.rand()

0.15552874999327126

In [12]:

class GeneticAlgorithm:
	"""
	Classe pour le recuit simulé	
	"""
	def __init__(self, values, weights, mutation_ratio, nbr_generation, decimation_percent,  population_size=None, capacity=10):
		self.capacity = capacity
		self.objects = [((v, w), i) for i, (v, w) in enumerate(zip(values, weights))]

		self.mutation_ratio = mutation_ratio
		self.population = []
		self.nbr_generation = nbr_generation
		self.decimation_percent = decimation_percent
		self.current_generation = 0
		
		self.best_solution = None
		self.best_value = 0

		if len(values) >= 10 :
			if population_size:
				self.population_size = population_size
			else:
				self.population_size = 1000
		else:
			self.population_size = int(2**len(values)*0.8)

			


	def evaluate_values(self, solution):
		"""
		solution : list of object index
		objects : list of tuples (value, weight)
		"""
		val = 0
		for i, obj in enumerate(solution):
			val += obj * self.objects[i][0][0]
		return val

	def evalute_weight(self, solution):
		"""
		solution : binairie number
		objects : list of tuples (value, weight)
		"""
		weight = 0
		for i, obj in enumerate(solution):
			weight += obj *self.objects[i][0][1]
		return weight
	
	def generate_population(self, size):
		for i in range(size):
			individual = [ np.random.randint(0, 2) for _ in range(len(self.objects))]
			while self.evalute_weight(individual) > self.capacity:
				individual = [ np.random.randint(0, 2) for _ in range(len(self.objects))]
			self.population.append(individual)
			

	def evaluation(self):
		""""
		Retourne les indices des indice trie par ordre decroissant
		"""""
		scores = []
		for individual in self.population:
			scores.append(self.evaluate_values(individual))
		scores = np.argsort(scores)[::-1]
		return scores[:int(self.decimation_percent*len(scores))]
	


	def selection(self):
		self.population = [self.population[i] for i in self.evaluation()]

	def regeneration(self):
		"""
		Regenerer la population
		"""
		nbr_crossover = int(self.mutation_ratio*self.population_size)
		self.crossover(nbr_crossover)
		self.current_generation += 1


	def crossover(self, nbr_crossover):
		"""
		Choisir deux parents parmis les meilleurs et faire un crossover
		"""
		children = []
		for _ in range(nbr_crossover):
			parent1 = self.population[np.random.randint(0, len(self.population))]
			parent2 = self.population[np.random.randint(0, len(self.population))]

			crossover_point = np.random.randint(0, min(len(parent1), len(parent2)))
			child = parent1[:crossover_point] + parent2[crossover_point:]
			while self.evalute_weight(child) > self.capacity:
				crossover_point = np.random.randint(0, min(len(parent1), len(parent2)))

				child = parent1[:crossover_point] + parent2[crossover_point:]
				child = self.mutation(child)
			children.append(child)	
		self.population += children
		

	def mutation(self, child):
		p = np.random.rand()
		muted_child = child[::]
		if p <= 0.1:
			indice_mutation = np.random.randint(0, len(child))
			muted_child[indice_mutation] = 1 - child[indice_mutation]
			while self.evalute_weight(child) > self.capacity:
				indice_mutation = np.random.randint(0, len(child))
				muted_child[indice_mutation] = 1 - child[indice_mutation]
		return muted_child
	
	def solve(self):
		self.generate_population(self.population_size)
		while self.current_generation < self.nbr_generation:
			self.selection()
			self.regeneration()
		
		
		return self.population[np.argmax([self.evaluate_values(individual) for individual in self.population])]



capacity = 10
values = [40, 50, 100, 95, 30]
weights = [2, 3, 1, 5, 3]

algo_gen = GeneticAlgorithm(values, weights, 0.1, 50, 0.5, capacity)
solution = algo_gen.solve()
print(solution)
print(algo_gen.evaluate_values(solution))
print(algo_gen.evalute_weight(solution))

[0, 1, 1, 1, 0]
245
9


# Testing on  datasets


In [53]:
import tabulate

## low dimensional problems

In [56]:
low_dimensional_path = "./instances_01_KP/low-dimensional"
low_dimensional_optimum_path = "./instances_01_KP/low-dimensional-optimum"
probleme_instances_name = os.listdir("./instances_01_KP/low-dimensional")
probleme_instances = dict({ probleme_instance : np.loadtxt(low_dimensional_path + f"/{probleme_instance}") for probleme_instance in probleme_instances_name})
probleme_instances

{'f2_l-d_kp_20_878': array([[ 20., 878.],
        [ 44.,  92.],
        [ 46.,   4.],
        [ 90.,  43.],
        [ 72.,  83.],
        [ 91.,  84.],
        [ 40.,  68.],
        [ 75.,  92.],
        [ 35.,  82.],
        [  8.,   6.],
        [ 54.,  44.],
        [ 78.,  32.],
        [ 40.,  18.],
        [ 77.,  56.],
        [ 15.,  83.],
        [ 61.,  25.],
        [ 17.,  96.],
        [ 75.,  70.],
        [ 29.,  48.],
        [ 75.,  14.],
        [ 63.,  58.]]),
 'f9_l-d_kp_5_80': array([[ 5., 80.],
        [33., 15.],
        [24., 20.],
        [36., 17.],
        [37.,  8.],
        [12., 31.]]),
 'f3_l-d_kp_4_20': array([[ 4., 20.],
        [ 9.,  6.],
        [11.,  5.],
        [13.,  9.],
        [15.,  7.]]),
 'f4_l-d_kp_4_11': array([[ 4., 11.],
        [ 6.,  2.],
        [10.,  4.],
        [12.,  6.],
        [13.,  7.]]),
 'f7_l-d_kp_7_50': array([[ 7., 50.],
        [70., 31.],
        [20., 10.],
        [39., 20.],
        [37., 19.],
        [ 7.,  4.]

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
low_dimensional_path = "./instances_01_KP/low-dimensional"
low_dimensional_optimum_path = "./instances_01_KP/low-dimensional-optimum"
probleme_instances_name = os.listdir("./instances_01_KP/low-dimensional")
probleme_instances = dict({ probleme_instance : np.loadtxt(low_dimensional_path + f"/{probleme_instance}") for probleme_instance in probleme_instances_name})
probleme_instances

results = []
for probleme_instance in probleme_instances:
    val = np.loadtxt(low_dimensional_optimum_path + f"/{probleme_instance}")
    instance_data = probleme_instances[probleme_instance]
    n, C = instance_data[0]
    
    r = instance_data[1:, 0]
    w = instance_data[1:, 1]
    
	


    mean_best_value = 0
    best_value = 0
    for i in range(50):
        ga = GeneticAlgorithm(r, w, 0.1, 50, 0.5, 1000, capacity=C)
        ga.solve()
        if ga.best_value > best_value:
            best_value = ga.best_value
        mean_best_value += ga.best_value
    mean_best_value /= len(range(50))
    results.append([f"{probleme_instance} : ", mean_best_value, best_value, val])

table =  tabulate.tabulate(results, headers=["nom probleme", "cout moyenne obtenue", "meilleur coup obtenue", "coup attendue"])
print(table)

KeyboardInterrupt: 

In [ ]:
with ProcessPoolExecutor(10) as p:
    

## Large optimum 

In [ ]:
def load_large_problem(file_name):
	with open(file_name) as f:
		lines = f.readlines()
		num_lines = len(lines)
		return np.loadtxt(file_name, max_rows=num_lines -1)
		


large_scale_path = "./instances_01_KP/large_scale"
large_scale_optimum_path = "./instances_01_KP/large_scale-optimum"
probleme_instances_name = os.listdir("./instances_01_KP/large_scale")
probleme_instances = dict({ probleme_instance : load_large_problem(large_scale_path + f"/{probleme_instance}") for probleme_instance in probleme_instances_name})

In [ ]:
from multiprocessing import Pool


def solve_problem(probleme_instance, solving_attempts=100):
	val = np.loadtxt(large_scale_optimum_path + f"/{probleme_instance}")
	instance_data = probleme_instances[probleme_instance]
	n, C = instance_data[0]

	r = instance_data[1:, 0]
	w = instance_data[1:, 1]

	mean_best_value = 0
	best_value = 0
	for i in range(solving_attempts):
		sa = GeneticAlgorithm(r, w, T0=1000,  capacity=C)
		sa.solve()
		if sa.best_value > best_value:
			best_value = sa.best_value
		mean_best_value += sa.best_value
	mean_best_value /= solving_attempts
	return [f"{probleme_instance} : ", mean_best_value, best_value, val]

with Pool(4) as p:
	results = p.map(solve_problem, probleme_instances_name)
table =  tabulate.tabulate(results, headers=["nom probleme", "cout moyenne obtenue", "meilleur coup obtenue", "coup attendue"])
print(table)

nom probleme               cout moyenne obtenue    meilleur coup obtenue    coup attendue
-----------------------  ----------------------  -----------------------  ---------------
knapPI_1_1000_1000_1 :                  4526.4                      7214            54503
knapPI_3_500_1000_1 :                   2661.46                     3409             7117
knapPI_3_10000_1000_1 :                58967.5                     60563           146919
knapPI_3_2000_1000_1 :                 11400.5                     12243            28919
knapPI_2_2000_1000_1 :                  9737.87                    10630            18051
knapPI_3_100_1000_1 :                   1119.53                     1624             2397
knapPI_1_10000_1000_1 :                49015.4                     58473           563647
knapPI_2_200_1000_1 :                   1059.65                     1256             1634
knapPI_1_200_1000_1 :                   1153.29                     2433            11238
knapPI_3_2

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
instance_names = [r[0] for r in results]
best_values = [r[2] for r in results]
theoritical_values = [r[3] for r in results]

errors = [abs(best - theoritical) for best, theoritical in zip(best_values, theoritical_values)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=instance_names, y=theoritical_values,
    mode='markers',  # Mode 'markers' pour afficher des points
    name='Données',
    error_y=dict(
        type='data',  # Type d'erreur : 'data' pour des erreurs spécifiques
        array=errors,  # Valeurs des erreurs
        visible=True,  # Rendre les barres d'erreur visibles
        color='red',  # Couleur des barres d'erreur
        thickness=1.5,  # Épaisseur des barres d'erreur
        width=5  # Largeur des barres d'erreur
    ),
    marker=dict(
        color='blue',  # Couleur des points
        size=10  # Taille des points
    )
))

# Personnalisation du layout
fig.update_layout(
    title='Scatter Plot avec Barres d\'Erreur',
    xaxis_title='Axe X',
    yaxis_title='Axe Y',
    template='plotly_white'  # Thème du graphique
)

# Affichage du graphique
fig.show()